In [ ]:
from PIL import Image, ImageDraw
import numpy as np
import glob
import pandas as pd
import bar_chart_race as bcr


In [ ]:
#crop all images in the folder
def cropImage():
    
    
    image_list = []
    croppedImage=[]

    for filename in glob.glob('D://DatenFürInsta//Bilder//topAnime//*.png'):
        print(filename)
        img = Image.open(filename)
        image_list.append(img)
        
    for img in image_list:
        # Open the input image as numpy array, convert to RGB
        npImage=np.array(img)
        h,w=img.size

        # Create same size alpha layer with circle
        alpha = Image.new('L', img.size,0)
        draw = ImageDraw.Draw(alpha)
        draw.pieslice([0,0,h,w],0,360,fill=255)

        # Convert alpha Image to numpy array
        npAlpha=np.array(alpha)

        # Add alpha layer to RGB
        npImage=np.dstack((npImage,npAlpha))

        # Save with alpha
        croppedImage.append(npImage)
    
        for (i, new) in enumerate(croppedImage):
            
           Image.fromarray(new).save('{}{}{}'.format('D://DatenFürInsta//Bilder//topAnimeFinal//', i+1, '.png'))

        
#resize the images by changing the params in submethod resize        
def resizeImage():
    image_list = []
    resized_images = []

    for filename in glob.glob('D://DatenFürInsta//Bilder//topAnimeFinal//*.png'):
        print(filename)
        img = Image.open(filename)
        image_list.append(img)

    for image in image_list:
        image = image.resize((420, 420))
        resized_images.append(image)

    for (i, new) in enumerate(resized_images):
        new.save('{}{}{}'.format('D://DatenFürInsta//Bilder//topAnimeFinal//', i+1, '.png'))

In [ ]:

#transpose the dataset to get the date as index
def transposeFile(pathFrom, pathTo):
    df=pd.read_excel(pathFrom)
    df=df.T
    df =df.reset_index(level=0)
    df.columns=df.iloc[0]
    df=df.drop(df.index[0])
    return df.to_csv(pathTo)

#change columny types but integer to float 
def readFile(pathFrom):
    df=pd.read_csv(pathFrom, thousands=',')
    cols = df.select_dtypes(exclude=['int']).columns
    df[cols] = df[cols].apply(pd.to_numeric, downcast='float', errors='coerce')
    return df



def period_summary(values, ranks):
    top2 = values.nlargest(2)
    leader = top2.index[0]
    lead = top2.iloc[0] - top2.iloc[1]
    s = f'{leader} by {lead:,.0f} billion'
    return {'s': s, 'x': .99, 'y': .03, 'ha': 'right', 'size': 8}

# Top Animes Dataset

In [ ]:
transposeFile("Top10mostpopularanimes.xlsx", "Top10mostpopularanimes.csv")

In [ ]:
df=readFile("Top10mostpopularanimes.csv")

In [ ]:
df=df.set_index('Country Name')
df=df.interpolate(method='linear', axis=0).bfill()
df=df.drop(columns=['Unnamed: 0'])

In [ ]:
#rename columns that have not supported signs 
dfAnime=dfAnime.rename(columns={"Naruto: Shippuden": "Naruto Shippuden", "Code Geass: HNL": "Code Geass HNL", "Code Geass: HNL R2": "Code Geass HNL R2",
                        "Fullmetal Alchemist: B": "Fullmetal Alchemist B"
                       
                       
                       
                       }
              
              
              )

In [ ]:
#get the names of all columns
cols=df.columns
#change its type to integer
df=df[cols].applymap(np.int64)

In [ ]:
#create a race bar chart




bcr.bar_chart_race(df=dfAnime,
        n_bars=8,
        img_label_folder='images_folder',
        tick_image_mode='trailing',
        fixed_order=False, 
        fixed_max=True, 
        steps_per_period=20, 
        period_length=2000,
        period_summary_func=period_summary,
        period_label={'x': .99, 'y': .1, 'ha': 'right', 'color': 'rebeccapurple'},
        title='Die Beliebtesten Animes',
        shared_fontdict={'family': 'Dejavu Sans', 'weight': 'bold',
                                    'color': 'rebeccapurple'},
        filter_column_colors=True,
        #bar_textposition='inside',
        bar_texttemplate='{x:,.0f}', 
        bar_label_font=7, 
        tick_label_font=7, 
        tick_template='{x:,.0f}',
        scale='linear', 
        fig=None, 
        writer=None, 
        fig_kwargs={'dpi': 144},
        period_template='Jahr {x:.0f}',
        colors='dark12',
        filename='topAnimesGer.mp4',
        
                  
                )
